# Extra Hacker Credit

Create a version of stream logger code that will work even if there are files from multiple streams and where each stream gets its own Excel file. For example, instead of just filenames starting with “BCM” (the stream identifier), there were other filenames starting with “JEF”, “MAE”, and more. Make sure that each csv gets put into the correct xlsx file. 

In [1]:
import pandas as pd
from pathlib import Path
from openpyxl import load_workbook, Workbook
from datetime import datetime

# Function to convert string to desired Excel datetime format
def convert_to_excel_datetime(date_str):
    try:
        dt = datetime.strptime(date_str, "%m/%d/%Y %H:%M")
        return dt
    except ValueError:
        return None

# Function to create or load a workbook
def get_or_create_workbook(file_path):
    if Path(file_path).exists():
        workbook = load_workbook(file_path)
    else:
        workbook = Workbook()
        workbook.save(file_path)
    return workbook

# Define the path to the logs folder
logs_path = Path('logs')

# Dictionary to keep track of the writer objects for each stream
writers = {}

# Step 1: Consolidate CSV files into Excel workbooks
for csv_file in logs_path.glob('*.csv'):
    # Extract stream identifier from the filename
    stream_id = csv_file.stem.split('-')[0]
    excel_file = f'{stream_id}.xlsx'
    
    # Create or get the existing writer for this stream
    if stream_id not in writers:
        workbook = get_or_create_workbook(excel_file)
        writers[stream_id] = pd.ExcelWriter(excel_file, engine='openpyxl', mode='a', if_sheet_exists='new')
    
    # Read the CSV file into a DataFrame
    df = pd.read_csv(csv_file, header=None, names=['datetime', 'scale', 'temperature'])
    
    # Write the DataFrame to a sheet named after the CSV file (without the .csv extension)
    sheet_name = csv_file.stem
    df.to_excel(writers[stream_id], sheet_name=sheet_name, index=False)

# Save all the writers
for writer in writers.values():
    writer.close()

# Step 2: Add formulas to each sheet in all workbooks
for stream_id in writers.keys():
    excel_file = f'{stream_id}.xlsx'
    workbook = load_workbook(excel_file)
    
    for sheet_name in workbook.sheetnames:
        if sheet_name != 'Sheet':  # Skip the initial blank sheet
            sheet = workbook[sheet_name]
            max_row = sheet.max_row

            # Convert datetime strings to Excel datetime objects
            for row in range(2, max_row + 1):
                cell_value = sheet[f'A{row}'].value
                excel_datetime = convert_to_excel_datetime(cell_value)
                if excel_datetime:
                    sheet[f'A{row}'].value = excel_datetime
            
            # Add labels for the formulas
            sheet['G2'] = 'Min Temperature'
            sheet['G3'] = 'Max Temperature'
            sheet['G4'] = 'Average Temperature'
            sheet['G6'] = 'Min Datetime'
            sheet['G7'] = 'Max Datetime'

            # Add the formulas
            sheet['H2'] = f"=round(MIN(C2:C{max_row}), 1)"
            sheet['H3'] = f"=round(MAX(C2:C{max_row}), 1)"
            sheet['H4'] = f"=round(AVERAGE(C2:C{max_row}), 1)"
            sheet['H6'] = f"=TEXT(MIN(A2:A{max_row}), \"yyyy-mm-dd hh:mm:ss\")"
            sheet['H7'] = f"=TEXT(MAX(A2:A{max_row}), \"yyyy-mm-dd hh:mm:ss\")"
            
            # Adjust column widths
            sheet.column_dimensions['A'].width = 18
            sheet.column_dimensions['G'].width = 18
            sheet.column_dimensions['H'].width = 18

    # Save the workbook with the formulas
    workbook.save(excel_file)

This above helps to save file of any stream identifier.